<a href="https://colab.research.google.com/github/Mortality-Prediction/Mortality-Calc/blob/main/Mortality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 336.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 702.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existin

In [ ]:
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# Define a function to load data and prepare models
def load_data_and_train_models(file):
    # Load and prepare the data
    data = pd.read_csv(file)
    X = data.drop('Status', axis=1)
    y = data['Status']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train models with scaled data
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_scaled, y_train)

    xgb_model = XGBClassifier(n_estimators=100, random_state=42)
    xgb_model.fit(X_train_scaled, y_train)

    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train_scaled, y_train)

    return imputer, scaler, rf_model, xgb_model, lr_model, X.columns

# Define a function to make predictions
def predict_mortality(file, **kwargs):
    imputer, scaler, rf_model, xgb_model, lr_model, columns = load_data_and_train_models(file)

    input_df = pd.DataFrame([kwargs], columns=columns)
    # Handle missing values in user input
    input_imputed = imputer.transform(input_df)
    input_scaled = scaler.transform(input_imputed)

    rf_prediction = rf_model.predict(input_scaled)[0]
    xgb_prediction = xgb_model.predict(input_scaled)[0]
    lr_prediction = lr_model.predict(input_scaled)[0]

    return {
        "Random Forest Prediction": 'Deceased' if rf_prediction == 1 else 'Survived',
        "XGBoost Prediction": 'Deceased' if xgb_prediction == 1 else 'Survived',
        "Logistic Regression Prediction": 'Deceased' if lr_prediction == 1 else 'Survived'
    }

# Create a Gradio interface
def get_gradio_interface():
    # Define default inputs based on column names
    input_columns = load_data_and_train_models('/content/Book2.csv')[6]  # get feature names
    inputs = [gr.inputs.Number(label=feature, default=0) for feature in input_columns]
    outputs = gr.outputs.Label(num_top_classes=3)

    return gr.Interface(fn=predict_mortality, inputs=inputs, outputs=outputs, live=True)

# Launch the interface
interface = get_gradio_interface()
interface.launch(share=True)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Book2.csv'

In [ ]:
from google.colab import files

uploaded = files.upload()


Saving Book2.csv to Book2.csv


In [ ]:
import os

# List files in the /content/ directory
os.listdir('/content/')


['.config', 'Book2.csv', 'sample_data']

In [ ]:
file_path = '/content/Book2.csv'


In [ ]:
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from google.colab import files

# Upload file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get the file name

# Define a function to load data and prepare models
def load_data_and_train_models(file):
    # Load and prepare the data
    data = pd.read_csv(file)
    X = data.drop('Status', axis=1)
    y = data['Status']

    # Handle missing values in training data
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

    # Scale the data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train models
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_model.fit(X_train_scaled, y_train)

    xgb_model = XGBClassifier(n_estimators=100, random_state=42)
    xgb_model.fit(X_train_scaled, y_train)

    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train_scaled, y_train)

    # Return imputer, scaler, models, and feature names
    return imputer, scaler, rf_model, xgb_model, lr_model, X.columns.tolist()

# Define a function to make predictions
def predict_mortality(file, **kwargs):
    imputer, scaler, rf_model, xgb_model, lr_model, columns = load_data_and_train_models(file)

    # Check if columns match
    input_df = pd.DataFrame([kwargs], columns=columns)
    # Handle missing values in user input
    input_imputed = imputer.transform(input_df)
    input_scaled = scaler.transform(input_imputed)

    rf_prediction = rf_model.predict(input_scaled)[0]
    xgb_prediction = xgb_model.predict(input_scaled)[0]
    lr_prediction = lr_model.predict(input_scaled)[0]

    return {
        "Random Forest Prediction": 'Deceased' if rf_prediction == 1 else 'Survived',
        "XGBoost Prediction": 'Deceased' if xgb_prediction == 1 else 'Survived',
        "Logistic Regression Prediction": 'Deceased' if lr_prediction == 1 else 'Survived'
    }

# Create a Gradio interface
def get_gradio_interface():
    # Define default inputs based on column names
    imputer, scaler, rf_model, xgb_model, lr_model, columns = load_data_and_train_models(file_path)  # Get feature names
    inputs = [gr.Number(label=feature) for feature in columns]  # Use gr.Number without default
    outputs = gr.Label()  # or gr.outputs.Label() in older versions

    return gr.Interface(fn=lambda **kwargs: predict_mortality(file_path, **kwargs), inputs=inputs, outputs=outputs, live=True)

# Launch the interface
interface = get_gradio_interface()
interface.launch(share=True)


Saving Book2.csv to Book2 (1).csv


/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1002: UserWarning: Expected 0 arguments for function <function get_gradio_interface.<locals>.<lambda> at 0x7fdca6fafb50>, received 14.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1010: UserWarning: Expected maximum 0 arguments for function <function get_gradio_interface.<locals>.<lambda> at 0x7fdca6fafb50>, received 14.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8e90067231289c9d4c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import pandas as pd
from google.colab import files

# Upload file
uploaded = files.upload()
file_path = list(uploaded.keys())[0]  # Get the file name

# Load and display the data
try:
    data = pd.read_csv(file_path)
    print(data.head())  # Display the first few rows
except Exception as e:
    print(f"Error loading file: {e}")


IndexError: list index out of range

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

def load_data_and_train_models(file):
    try:
        # Load and prepare the data
        data = pd.read_csv(file)
        X = data.drop('Status', axis=1)
        y = data['Status']

        # Handle missing values
        imputer = SimpleImputer(strategy='mean')
        X_imputed = imputer.fit_transform(X)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)

        # Scale the data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Train models
        rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_model.fit(X_train_scaled, y_train)

        xgb_model = XGBClassifier(n_estimators=100, random_state=42)
        xgb_model.fit(X_train_scaled, y_train)

        lr_model = LogisticRegression(max_iter=1000)
        lr_model.fit(X_train_scaled, y_train)

        return imputer, scaler, rf_model, xgb_model, lr_model, X.columns.tolist()

    except Exception as e:
        return str(e), None, None, None, None, None

# Test data loading and model training
imputer, scaler, rf_model, xgb_model, lr_model, columns = load_data_and_train_models(file_path)
print(f"Features: {columns}")


Features: ['subject_id', 'Respiratory Rate', 'Bilirubin', 'Creatinine', 'DBP (noninvasive)', 'DBP', 'HR', 'Lactate', 'Mean Arterial Pressure', 'Mean Blood Pressure (noninvasive)', 'Mean Blood Pressure', 'SBP (noninvasive)', 'SBP', 'Temperature']


In [ ]:
import gradio as gr

def predict_mortality(**kwargs):
    try:
        # Use the same file path and models
        imputer, scaler, rf_model, xgb_model, lr_model, columns = load_data_and_train_models(file_path)

        if not isinstance(columns, list):  # Check if data loading failed
            return {"Error": imputer}

        # Check if columns match
        input_df = pd.DataFrame([kwargs], columns=columns)
        input_imputed = imputer.transform(input_df)
        input_scaled = scaler.transform(input_imputed)

        rf_prediction = rf_model.predict(input_scaled)[0]
        xgb_prediction = xgb_model.predict(input_scaled)[0]
        lr_prediction = lr_model.predict(input_scaled)[0]

        return {
            "Random Forest Prediction": 'Deceased' if rf_prediction == 1 else 'Survived',
            "XGBoost Prediction": 'Deceased' if xgb_prediction == 1 else 'Survived',
            "Logistic Regression Prediction": 'Deceased' if lr_prediction == 1 else 'Survived'
        }
    except Exception as e:
        return {"Error": str(e)}

def get_gradio_interface():
    try:
        # Load data and models to get feature names
        imputer, scaler, rf_model, xgb_model, lr_model, columns = load_data_and_train_models(file_path)

        if not isinstance(columns, list):  # Check if data loading failed
            return gr.Interface(
                fn=lambda *args, **kwargs: {"Error": imputer},
                inputs=[],
                outputs="text",
                title="Mortality Rate Calculator"
            )

        # Create Gradio inputs based on feature names
        inputs = [gr.Number(label=feature) for feature in columns]
        outputs = gr.JSON()  # Output predictions as JSON

        return gr.Interface(
            fn=lambda **kwargs: predict_mortality(**kwargs),
            inputs=inputs,
            outputs=outputs,
            title="Mortality Rate Calculator"
        )
    except Exception as e:
        return gr.Interface(
            fn=lambda *args, **kwargs: {"Error": str(e)},
            inputs=[],
            outputs="text",
            title="Mortality Rate Calculator"
        )

# Launch the interface
interface = get_gradio_interface()
interface.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1002: UserWarning: Expected 0 arguments for function <function get_gradio_interface.<locals>.<lambda> at 0x7fdca6fafc70>, received 14.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1010: UserWarning: Expected maximum 0 arguments for function <function get_gradio_interface.<locals>.<lambda> at 0x7fdca6fafc70>, received 14.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://00309f3fe3cf27b1a4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
